<span style="color: rgb(34, 34, 34); font-family: -system-ui, -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Roboto, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;, &quot;Segoe UI Symbol&quot;; font-size: 20px; font-weight: bold; text-align: center;">Pricing and Ratings</span>

### **1. <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">If a Meat Lovers pizza costs $12 and Vegetarian costs $10 and there were no charges for changes - how much money has Pizza Runner made so far if there are no delivery fees?</span>**

Ans - Used a case to alot $12 to meatlovers pizza and $10 to Vegetarian pizza  for all the delivered orders. At last used sum to calculate total sales.

In [7]:
with total_money_made as(
select a.order_id,a.pizza_id,pizza_name, 
CASE
WHEN pizza_name='Meatlovers' then 12
WHEN pizza_name='Vegetarian' then 10
end as cost
from customer_orders a inner join runner_orders b on a.order_id = b.order_id 
inner join pizza_names c on a.pizza_id = c.pizza_id where cancellation not like '%Cancellation%'
)
select sum(cost) as money_made_in_dollars from total_money_made

(1 row affected)

Total execution time: 00:00:00.026

money_made_in_dollars
138


### **2. <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">What if there was an additional $1 charge for any pizza extras?</span>**

- <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);"><b>Add cheese is $1 extra</b></span>

Ans -

In [12]:
with sucessful_customer_orders as(
select a.order_id,pizza_id,extras from customer_orders a inner join runner_orders b on a.order_id = b.order_id where cancellation not like '%Cancellation%'
),
not_null_extras as(
select * from sucessful_customer_orders where extras <> ''
),
topping_cost as(
select cast(trim(value) as int) as topping,
case
when cast(trim(value) as int) is not null then 1
end as topping_cost
from not_null_extras
cross apply string_split(not_null_extras.extras, ',') where extras <> 'null'
)
select sum(topping_cost) from topping_cost

(1 row affected)

Total execution time: 00:00:00.036

(No column name)
4


### **3. <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">What if substitutes were allowed at no additional cost but any additional extras were charged at $1</span>**

- <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);"><b>Exclude Cheese and add Bacon is free</b></span>
- <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);"><b>Exclude Cheese but add bacon and beef costs $1 extra</b></span>

In [39]:
with sucessful_customer_orders as(
select a.order_id,pizza_id,extras,exclusions from customer_orders a inner join runner_orders b on a.order_id = b.order_id where cancellation not like '%Cancellation%'
),
new_table as (
select order_id,pizza_id,trim([value]) as segreggated_extras,exclusions from sucessful_customer_orders
cross apply string_split(sucessful_customer_orders.extras,',')
),
new_table_1 as (
select order_id,pizza_id,segreggated_extras,trim([value]) as seggregated_exclusions from new_table
cross apply string_split(new_table.exclusions,',')
),
new_table_2 as(
select order_id,pizza_id,
case
when segreggated_extras is null then 0
when segreggated_extras = 'null' then 0
when segreggated_extras='' then 0
else cast(segreggated_extras as int)
end as new_segreggated_extras,
case
when seggregated_exclusions is null then 0
when seggregated_exclusions = 'null' then 0
when seggregated_exclusions='' then 0
else cast(seggregated_exclusions as int)
end as new_seggregated_exclusions
from new_table_1
),
new_table3 as (
select order_id,pizza_id,
case
when new_segreggated_extras <>0 then 1
else 0
end as count_of_extras,
case
when new_seggregated_exclusions<>0 then 1
else 0
end as count_of_exclusions
from new_table_2
)
select sum(count_of_extras-count_of_exclusions) from new_table3 where (count_of_extras-count_of_exclusions) > 0

(1 row affected)

Total execution time: 00:00:00.033

(No column name)
2


4. <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; font-weight: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">What if meat substitutes and vegetable substitutes were allowed but any change outside were charged at $2 and $1 respectively?</span>

- <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; font-weight: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">Exclude Cheese and add Bacon is $2 extra</span>
- <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; font-weight: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">Exclude Beef and add mushroom is $1 extra</span>
- <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; font-weight: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">Exclude Beef and add Bacon is free</span>
- <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; font-weight: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">Exclude Beef and Mushroom, and add Bacon and Cheese is free</span>

In [52]:
with topping_type as (
select topping_id,topping_name,
case 
when topping_id = 1 or topping_id =3 or topping_id =5 or topping_id =8 or topping_id =10 then 'Meat'
else 'Vegetable'
end as substitute
from pizza_toppings
),
sucessful_customer_orders as(
select a.order_id,pizza_id,extras,exclusions from customer_orders a inner join runner_orders b on a.order_id = b.order_id where cancellation not like '%Cancellation%'
),
new_table as (
select order_id,pizza_id,trim([value]) as segreggated_extras,exclusions from sucessful_customer_orders
cross apply string_split(sucessful_customer_orders.extras,',')
),
new_table_1 as (
select order_id,pizza_id,segreggated_extras,trim([value]) as seggregated_exclusions from new_table
cross apply string_split(new_table.exclusions,',')
),
new_table_2 as(
select order_id,pizza_id,
case
when segreggated_extras is null then 0
when segreggated_extras = 'null' then 0
when segreggated_extras='' then 0
else cast(segreggated_extras as int)
end as new_segreggated_extras,
case
when seggregated_exclusions is null then 0
when seggregated_exclusions = 'null' then 0
when seggregated_exclusions='' then 0
else cast(seggregated_exclusions as int)
end as new_seggregated_exclusions
from new_table_1
),
new_table3 as(
select order_id,pizza_id,new_segreggated_extras,b.substitute as extras_substitute,new_seggregated_exclusions,c.substitute as exclusions_substitute
from new_table_2 a left join topping_type b 
on a.new_segreggated_extras = b.topping_id left join topping_type c on a.new_seggregated_exclusions = c.topping_id
),
row_number_count as (
Select *,
ROW_NUMBER() OVER(Partition by order_id,pizza_id,new_segreggated_extras order by order_id) as dense_rank
FROM new_table3
)
select * from row_number_count where dense_rank = 1

(11 rows affected)

Total execution time: 00:00:00.135

order_id,pizza_id,new_segreggated_extras,extras_substitute,new_seggregated_exclusions,exclusions_substitute,dense_rank
4,1,0,NULL,4,Vegetable,1
10,1,1,Meat,2,Vegetable,1
10,1,4,Vegetable,6,Vegetable,1
1,1,0,NULL,0,NULL,1
2,1,0,NULL,0,NULL,1
3,1,0,NULL,0,NULL,1
4,2,0,NULL,4,Vegetable,1
5,1,1,Meat,0,NULL,1
7,2,1,Meat,0,NULL,1
8,1,0,NULL,0,NULL,1


### **5. <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">The Pizza Runner team now wants to add an additional ratings system that allows customers to rate their runner, how would you design an additional table for this new dataset - generate a schema for this new table and insert your own data for ratings for each successful customer order between 1 to 5.</span>**

In [1]:
DROP TABLE IF EXISTS ratings_system;
CREATE TABLE ratings_system (
  "order_id" INTEGER,
  "runner_id" INTEGER,
  "rating" INTEGER
);

INSERT INTO ratings_system
  ("order_id","runner_id", "rating")
VALUES
  (1,1,4),
  (2,1,3),
  (3,1,4),
  (4,2,5),
  (5,3,3),
  (7,2,2),
  (8,2,4),
  (10,1,5);

(8 rows affected)

Total execution time: 00:00:00.132

### **6. <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">Using your newly generated table - can you join all of the information together to form a table which has the following information for successful deliveries?</span>**

- <span style="background-color: rgb(250, 250, 250); color: rgb(255, 85, 74); font-family: &quot;Bitstream Vera Sans Mono&quot;, Courier, monospace; font-size: 16px; font-style: inherit; font-variant-caps: inherit; caret-color: rgb(64, 64, 64);"><b>customer_id</b></span>

- <span style="background-color: rgb(250, 250, 250); color: rgb(255, 85, 74); font-family: &quot;Bitstream Vera Sans Mono&quot;, Courier, monospace; font-size: 16px; font-style: inherit; font-variant-caps: inherit; caret-color: rgb(64, 64, 64);"><b>order_id</b></span>

- <span style="background-color: rgb(250, 250, 250); color: rgb(255, 85, 74); font-family: &quot;Bitstream Vera Sans Mono&quot;, Courier, monospace; font-size: 16px; font-style: inherit; font-variant-caps: inherit; caret-color: rgb(64, 64, 64);"><b>runner_id</b></span>

- <span style="background-color: rgb(250, 250, 250); color: rgb(255, 85, 74); font-family: &quot;Bitstream Vera Sans Mono&quot;, Courier, monospace; font-size: 16px; font-style: inherit; font-variant-caps: inherit; caret-color: rgb(64, 64, 64);"><b>rating</b></span>

- <span style="background-color: rgb(250, 250, 250); color: rgb(255, 85, 74); font-family: &quot;Bitstream Vera Sans Mono&quot;, Courier, monospace; font-size: 16px; font-style: inherit; font-variant-caps: inherit; caret-color: rgb(64, 64, 64);"><b>order_time</b></span>

- <span style="background-color: rgb(250, 250, 250); color: rgb(255, 85, 74); font-family: &quot;Bitstream Vera Sans Mono&quot;, Courier, monospace; font-size: 16px; font-style: inherit; font-variant-caps: inherit; caret-color: rgb(64, 64, 64);"><b>pickup_time</b></span>

- <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);"><b>Time between order and pickup</b></span>

- <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);"><b>Delivery duration</b></span>

- <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);"><b>Average speed</b></span>

- <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);"><b>Total number of pizzas</b></span>

In [4]:
select a.customer_id,a.order_id,b.runner_id,c.rating,a.order_time,b.pickup_time,cast((b.pickup_time-a.order_time)as time),b.duration from customer_orders a inner join runner_orders b on a.order_id = b.order_id inner join ratings_system c on a.order_id = c.order_id where b.cancellation not like '%Cancellation%'

(12 rows affected)

Total execution time: 00:00:00.076

customer_id,order_id,runner_id,rating,order_time,pickup_time,(No column name),duration
101,1,1,4,2021-01-01 18:05:02.000,2021-01-01 18:15:34.000,00:10:32,32 minutes
101,2,1,3,2021-01-01 19:00:52.000,2021-01-01 19:10:54.000,00:10:02,27 minutes
102,3,1,4,2021-01-02 23:51:23.000,2021-01-03 00:12:37.000,00:21:14,20 mins
102,3,1,4,2021-01-02 23:51:23.000,2021-01-03 00:12:37.000,00:21:14,20 mins
103,4,2,5,2021-01-04 13:23:46.000,2021-01-04 13:53:03.000,00:29:17,40
103,4,2,5,2021-01-04 13:23:46.000,2021-01-04 13:53:03.000,00:29:17,40
103,4,2,5,2021-01-04 13:23:46.000,2021-01-04 13:53:03.000,00:29:17,40
104,5,3,3,2021-01-08 21:00:29.000,2021-01-08 21:10:57.000,00:10:28,15
105,7,2,2,2021-01-08 21:20:29.000,2021-01-08 21:30:45.000,00:10:16,25mins
102,8,2,4,2021-01-09 23:54:33.000,2021-01-10 00:15:02.000,00:20:29,15 minute


### **7. <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">If a Meat Lovers pizza was $12 and Vegetarian $10 fixed prices with no cost for extras and each runner is paid $0.30 per kilometre traveled - how much money does Pizza Runner have left over after these deliveries?</span>**

In [9]:
select sum(dbo.udf_GetNumeric(distance)*.30) from runner_orders where cancellation not like '%Cancellation%'

(1 row affected)

Total execution time: 00:00:00.029

(No column name)
43.56


### **8. <span style="font-family: inherit; font-size: inherit; font-style: inherit; font-variant-caps: inherit; color: rgb(64, 64, 64); caret-color: rgb(64, 64, 64);">If 1 unit of each ingredient costs $0.50 - how much net revenue will Pizza Runner make if the costs from question 30 are used?</span>**

In [18]:
with newtable1 as(
select pizza_id,count(trim(value)) as topping_count from pizza_recipes
cross apply string_split(toppings, ',')
group by pizza_id
)
select sum(c.topping_count)*.50 from customer_orders a inner join runner_orders b on a.order_id = b.order_id inner join newtable1 c on a.pizza_id = c.pizza_id where cancellation not like '%Cancellation%'

(1 row affected)

Total execution time: 00:00:00.052

(No column name)
45.00


Total Revenue = 138 - 43.56 - 45

 <span style="background-color: rgba(127, 127, 127, 0.1); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">=$49.44</span>